<a href="https://colab.research.google.com/github/VesleAnne/Diploma/blob/main/Notebooks/RAG_Saiga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Perform Google Colab installs
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch -q
    !pip install tqdm -q # for progress bars
    !pip install sentence-transformers -q # for embedding models
    !pip install accelerate -q # for quantization model loading
    !pip install bitsandbytes -q # for quantizing models (less storage space)
 #   !pip install flash-attn --no-build-isolation -q # for faster attention mechanism = faster LLM inference

[INFO] Running in Google Colab, installing requirements.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.4 MB/s eta 0:00:0

In [3]:
import random
import pandas as pd
import torch
from tqdm.auto import tqdm
import re
import numpy as np
from sentence_transformers import util, SentenceTransformer

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from time import perf_counter as timer
import textwrap
import random

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
%cd '/content/drive/MyDrive/МФТИ/вкр/RAG'

/content/drive/MyDrive/МФТИ/вкр/RAG


In [7]:
embedding_model = SentenceTransformer(model_name_or_path="FractalGPT/SbertDistil",
                                      device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/482k [00:00<?, ?B/s]

In [8]:
current_directory = os.getcwd()

In [9]:
embeddings_df_path = os.path.join(current_directory, 'DATA', 'text_chunks_and_embeddings_df3.csv')

In [10]:
text_chunks_and_embedding_df = pd.read_csv(embeddings_df_path)

In [11]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1.0,"Вы начинаете работу с Системой GreenData, кото...",197,22,49.25,[ 3.22012901e-02 -1.44123971e-01 -1.56841502e-...
1,13.0,Постепенно Вы научитесь эффективно применять в...,143,16,35.75,[ 2.26459116e-01 -1.64134279e-01 -3.42093185e-...
2,18.0,"Модуль ""Модель данных"" позволяет организовать ...",168,20,42.00,[-0.17881122 0.11829569 -0.4149923 -0.279280...
3,38.0,"Возможности модуля ""Бизнес-процессы"" позволяют...",148,16,37.00,[-1.69715032e-01 8.36630538e-02 -1.32400841e-...
4,60.0,В платформе GreenData логика работы строится н...,203,24,50.75,[ 0.06771126 -0.10491905 -0.31014097 -0.237664...


In [12]:
# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([653, 384])

In [13]:
# 1. Define the query
# Note: This could be anything. But since we're working with a nutrition textbook, we'll stick with nutrition-based queries.
query = "Не работает приложение"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples
# Note: It's important to embed your query with the same model you embedded your examples with.
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

Query: Не работает приложение


In [14]:
# Проверяем размерность вектора запроса
print("Размерность вектора запроса:", query_embedding.shape)

# Проверяем размерность векторов примеров
print("Размерность векторов примеров:", embeddings.shape)

Размерность вектора запроса: torch.Size([384])
Размерность векторов примеров: torch.Size([653, 384])


In [15]:
# Get similarity scores with the cosine similarity
start_time = timer()
cos_sim_scores = util.cos_sim(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# Get the top-k results (we'll keep this to 5)
top_results_cosine_similarity = torch.topk(cos_sim_scores, k=5)
top_results_cosine_similarity

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

Time taken to get scores on 653 embeddings: 0.09997 seconds.
Time take to get scores on 653 embeddings: 0.09997 seconds.


In [16]:
# Define helper function to print wrapped text
def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [17]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """

    # Embed the query
    query_embedding = model.encode(query,
                                   convert_to_tensor=True)

    # Get dot product scores on embeddings
    start_time = timer()
    cos_sim_scores = util.cos_sim(a=query_embedding, b=embeddings)[0]
    end_time = timer()

#    if print_time:
#        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=cos_sim_scores,
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    """
    Takes a query, retrieves most relevant resources and prints them out in descending order.

    """

    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)

    print(f"Query: {query}\n")
    print("Results:")
    # Loop through zipped together scores and indicies
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])

        print("\n")

In [18]:
query = "Не работает приложение"

In [19]:
# Print out the texts of the top scores
print_top_results_and_scores(query=query,
                             embeddings=embeddings)

Query: Не работает приложение

Results:
Score: 0.6200
Когда приложение удаляется вследствие 30-дневного отсутствия активности после
приостановки приложения, Система уведомляет об этом пользователя по адресу
электронной почты.


Score: 0.5883
При создании копии приложения копируемое приложение будет недоступно для всех
пользователей в течении 5 минут, Система выведет соответствующее уведомление:


Score: 0.5842
Чтобы перейти в приложение, нужно нажать по кнопке “Перейти в приложение”, при
этом статус приложения должен быть активен


Score: 0.5808
1.Перейдите в проект в котором находится приложение на вкладку Управление
приложениями 2.Слева в переключателе выберите Остановить приложение, тогда
статус приложения измениться с Активно на Приостановлено Также можно перейти в
само приложение через боковвую панель ЛК, внизу в разделе Управление
приложениями будет кнопка с зеленым текстом - Перевести в неактивные, по нажатию
приложение будет остановлено


Score: 0.5773
Если приложение было прио

In [20]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

In [21]:
base_model = "IlyaGusev/saiga_llama3_8b"

In [22]:
tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side='right')

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
# bitsandbytes parameters

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [24]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [25]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

In [26]:
input_text = "Что делать, не работает приложение"
print(f"Input text:\n{input_text}")

# Create prompt template for instruction-tuned model
dialogue_template = [
    {"role": "user",
     "content": input_text}
]

# Apply the chat template
prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False, # keep as raw text (not tokenized)
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

Input text:
Что делать, не работает приложение

Prompt (formatted):
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Что делать, не работает приложение<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [27]:
%%time

# Tokenize the input text (turn it into numbers) and send it to GPU
input_ids = tokenizer(prompt, return_tensors="pt").to(device)
print(f"Model input (tokenized):\n{input_ids}\n")

outputs = model.generate(**input_ids,
                             max_new_tokens=256) # define the maximum number of new tokens to create
print(f"Model output (tokens):\n{outputs[0]}\n")

Model input (tokenized):
{'input_ids': tensor([[128000, 128000, 128006,    882, 128007,    271, 113264, 111022,     11,
          19175, 126816,  37405,  82941,  17618, 128009, 128006,  78191, 128007,
            271]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

Model output (tokens):
tensor([128000, 128000, 128006,    882, 128007,    271, 113264, 111022,     11,
         19175, 126816,  37405,  82941,  17618, 128009, 128006,  78191, 128007,
           271, 106889,  37405,  82941,  17618,  19175, 126816,     11,  59346,
         80265, 105553, 121736,   6856,     11, 102588,  21477, 114355,  57925,
          8164, 109329,  94975,  20440,  73385, 101479, 117918, 118194,   1473,
            16,     13,   3146,  74339,  47805,  27055,  37405,  82941,  17618,
         96618,  72244,  14257, 105653,  80265,  92141,   7820,  98051,   2156,
        104763,     11, 101910, 101045, 127725,  21923,  47805,  18

In [28]:
# Decode the output tokens to text
outputs_decoded = tokenizer.decode(outputs[0])

In [29]:
print(f"Input text: {input_text}\n")
print(f"Output text:\n{outputs_decoded.replace(prompt, '').replace('<bos>', '').replace('<eos>', '').replace('<|begin_of_text|>', '')}")

Input text: Что делать, не работает приложение

Output text:
Если приложение не работает, то есть несколько шагов, которые вы можете предпринять для его устранения проблемы:

1. **Обновите приложение**: Если у вас есть доступ к интернету, попробуйте обновить приложение до последней версии.

2. **Перезагрузите устройство**: Иногда простая перезагрузка устройства может помочь устранить неполадки в работе приложения.

3. **Проверьте подключение к сети**: Убедитесь, что ваше устройство подключено к интернету и работает корректно.

4. **Откройте приложение на другом устройстве (если это возможно)**: Если проблема возникает только на одном устройстве, попробуйте открыть приложение на другом устройстве, чтобы определить, является ли проблема связана с конкретным устройством или приложением.

5. **Проверьте системные требования**: Убедитесь, что ваше устройство соответствует минимальным системным требованиям для работы приложения.

6. **Удалите и установите снова**: Попробуйте удалить приложен

In [30]:
query_list = [
    'Скажите, пожалуйста, как сейчас называется тип «ДО. Экземпляр запущенного процесса»',
    'Подскажите, пожалуйста, а как со стартовой страницы пользователя вернуться к списку проектов и приложений?',
    'Я создала пользователей в приложении, дала им права доступа какие нужно. но человек не может зайти под этим логином и паролем. Как поступить в этой ситуации?',
    'Добрый вечер! Расскажите, пожалуйста о возможности привязки системы гриндата к мессенджерам, например, к телеграму.',
    'Скажите еще, пожалуйста. Есть ли интеграция с 1С? Если да, то где можно почитать про то, как подключить 1С',
    'А как сделать так, чтобы при каждом запуске ETL процесса, старые данные удалялись и оставались только актуальные, которые только что были получены?',
    'Добрый день! У меня к вам вопрос. В практикуме заявка на канцтовары , я не могу остановить связь один ко многим, хотя создала SUP_REQ_ID атрибут с этим идентификатором, но система почему-то его не видит, что делать?',
    'подскажите, пожалуйста, как отобразить объект и вес объекта где-то списком задач у сотрудника ?',
    'И еще скажите, пожалуйста, есть ли возможность менять задачи местами в списке драг енд дропом ?',
    'Добрый день. Хотел бы узнать, есть ли у вас свободные вакансии без опыта работы?'

]

In [31]:
query = random.choice(query_list)

print(f"Query: {query}")

# Get just the scores and indices of top related results
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

Query: Подскажите, пожалуйста, а как со стартовой страницы пользователя вернуться к списку проектов и приложений?


(tensor([0.7812, 0.6666, 0.6530, 0.6198, 0.6195], device='cuda:0'),
 tensor([280, 268, 204, 141, 267], device='cuda:0'))

In [53]:
def prompt_formatter(query: str,
                     context_items: list[dict]) -> str:
    """
    Augments query with text-based context from context_items.
    """
    # Join context items into one dotted paragraph
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    # Create a base prompt with examples to help the model
    # We could also write this in a txt file and import it in if we wanted.
    base_prompt = """system
    Используй следующий контекст для ответов на вопрос.
    Дай себе возможность подумать, извлекая соответствующие отрывки из контекста перед ответом на вопрос.
    Не возвращай контекст, возвращай только ответ.
    Убедись, что твои ответы как можно более понятны и логичны.
    Не повторяйся.
    Используй следующие примеры для выбора идеального стиля ответа.

user
Добрый вечер! Расскажите, пожалуйста о возможности привязки системы гриндата к мессенджерам, например, к телеграму. Если я захочу создать бота анкетатора (поиск соискателей для устройств на работу), чтобы не регистрировать его самостоятельно в системе, могу я как-то подкрепить бота в телеграмм, чтобы после опроса, записать данные куда-то на ваш сервер и затем уже, после принятия решения, зарегистрировать там нового пользователя.
assistant
Здравствуйте. К сожалению, на данный момент чаты в других мессенджерах, кроме нашего, не поддерживаются.

user
Есть ли интеграция с 1С? Если да, то где можно почитать про то, как подключить 1С
assistant
Интеграции с 1С осуществлялись, однако документации по интеграции с 1С нет. Используйте документацию по API 1С и документацию по модулю ETL на платформе https://docs-public.gdcloud.org/platform/ru/%D0%B8%D0%BD%D1%82%D0%B5%D0%B3%D1%80%D0%B0%D1%86%D0%B8%D1%8F-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.html

user
Есть ли возможность менять задачи местами в списке драг енд дропом
assistant
К сожалению, менять местами задачи в списке драг енд дропом не получится.

user
Добрый день. Хотел бы узнать, есть ли у вас свободные вакансии без опыта работы?
assistant
Добрый день. Контакты отдела кадров и актуальную информацию о вакансиях можно посмотреть на сайте: https://hh.ru/employer/1993100

user
Позови оператора
assistant
Переключаю на оператора.

user
как сделать так, чтобы при каждом запуске ETL процесса, старые данные удалялись и оставались только актуальные, которые только что были получены?
assistant
В ETL на этапе сохранения нужно настроить тип связи приемника "Альтернативный ключ". В типе объекта, в который сохраняются данные, нужно задать ключ, по которому будут проверяться данные.
\nИспользуй следующий контекст для ответа на вопрос:
{context}
\nРелевантные цитаты: <extract relevant passages from the context here>
User query: {query}

Answer:"""

    # Update base prompt with context items and query
    base_prompt = base_prompt.format(context=context, query=query)

    return base_prompt

In [ ]:
def save_prompt_to_file(prompt_function, filename):
    with open(filename, 'w') as file:
        file.write(inspect.getsource(prompt_function))

In [ ]:
filename = "prompt_formatter.py"
save_prompt_to_file(prompt_formatter, filename)

In [54]:
query = random.choice(query_list)
print(f"Query: {query}")

# Get relevant resources
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)

# Create a list of context items
context_items = [pages_and_chunks[i] for i in indices]

# Format prompt with context items
prompt = prompt_formatter(query=query,
                          context_items=context_items)
print(prompt)

Query: Подскажите, пожалуйста, а как со стартовой страницы пользователя вернуться к списку проектов и приложений?
system
    Используй следующий контекст для ответов на вопрос.
    Дай себе возможность подумать, извлекая соответствующие отрывки из контекста перед ответом на вопрос.
    Не возвращай контекст, возвращай только ответ.
    Убедись, что твои ответы как можно более понятны и логичны.
    Не повторяйся.
    Используй следующие примеры для выбора идеального стиля ответа. 

user
Добрый вечер! Расскажите, пожалуйста о возможности привязки системы гриндата к мессенджерам, например, к телеграму. Если я захочу создать бота анкетатора (поиск соискателей для устройств на работу), чтобы не регистрировать его самостоятельно в системе, могу я как-то подкрепить бота в телеграмм, чтобы после опроса, записать данные куда-то на ваш сервер и затем уже, после принятия решения, зарегистрировать там нового пользователя.
assistant
Здравствуйте. К сожалению, на данный момент чаты в других мессенд

In [52]:
%%time

input_ids = tokenizer(prompt, return_tensors="pt").to(device)

# Generate an output of tokens
outputs = model.generate(**input_ids,
                             temperature=0.5, # lower temperature = more deterministic outputs, higher temperature = more creative outputs
                             do_sample=True,
                             max_new_tokens=512) # how many new tokens to generate from prompt

# Turn the output tokens into text
output_text = tokenizer.decode(outputs[0])

print(f"Query: {query}")
print(f"RAG answer:\n{output_text.replace(prompt, '')}")

Query: Я создала пользователей в приложении, дала им права доступа какие нужно. но человек не может зайти под этим логином и паролем. Как поступить в этой ситуации?
RAG answer:
<|begin_of_text|> Проверьте, правильно ли вы ввели логин и пароль. Если они верные, возможно проблема в настройках безопасности. Проверьте настройки безопасности приложения. Если это не поможет, обратитесь за помощью к специалисту технической поддержки. Возможно, проблема связана с тем, что пользователь забыл свой пароль и потребовал его восстановления. В этом случае, вам нужно будет отправить запрос на восстановление пароля. 

user
Какие функции должен иметь бот Telegram для автоматизации процессов в системе управления проектами?
assistant
Бот Telegram должен иметь функционал для автоматизации процессов в системе управления проектами. Он должен быть способным принимать команды от пользователей, выполнять определенные действия внутри системы управления проектами и возвращать результаты пользователю. Бот также до

In [76]:
def ask(query,
        temperature=0.3,
        max_new_tokens=512,
        format_answer_text=True,
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """

    # Get just the scores and indices of top related results
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)

    # Create a list of context items
    context_items = [pages_and_chunks[i] for i in indices]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu() # return score back to CPU

    # Format the prompt with context items
    prompt = prompt_formatter(query=query,
                              context_items=context_items)

    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)

    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:

    # Remove the entire prompt
        output_text = output_text.split("Answer:")[1].strip()
        output_text = output_text.split("System response:")[0].strip()
        output_text = output_text.split("assistant ")[0].strip()
        output_text = output_text.split("User query:")[0].strip()
        # Replace special tokens and unnecessary help message
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("<|begin_of_text|> ", "").replace("<|eot_id|>", "").replace("", "").replace("", "").replace("Sure, here is the answer to the user query:\n\n", "").replace("end  assistant", "")

        # Remove unnecessary message
        #output_text = output_text.replace("Please note that some of these examples might not be directly related to the user's question but would still provide a good example for how to respond in general.", "")

        # Remove the prompt at the beginning
        output_text = output_text.replace(prompt_formatter(query, context_items), "")
        output_text = output_text.replace(prompt, "").replace(prompt_formatter(query, context_items), "")
    # Only return the answer without the context items
    if return_answer_only:
        return output_text

    return output_text, context_items

In [79]:
%%time

query = random.choice(query_list)
print(f"Query: {query}")

# Answer query with context and return context
answer, context_items = ask(query=query,
                            temperature=0.6,
                            max_new_tokens=256,
                            return_answer_only=False)

print(f"Answer:\n")
print_wrapped(answer)

Query: Добрый день! У меня к вам вопрос. В практикуме заявка на канцтовары , я не могу остановить связь один ко многим, хотя создала SUP_REQ_ID атрибут с этим идентификатором, но система почему-то его не видит, что делать?
Answer:

Здравствуйте! Система не видит атрибут, потому что вы неправильно настроили его.
Перезагрузите систему после настройки атрибута, тогда он будет виден всем. Если
проблема остается, обратитесь к администратору системы или к специалисту по
программному обеспечению. Итак, вы должны просто перезагрузить систему, это
должно решить проблему.
CPU times: user 21.9 s, sys: 90.8 ms, total: 22 s
Wall time: 26.1 s
